These finetuning steps are based on the tutorial linked here: https://medium.com/nlplanet/bert-finetuning-with-hugging-face-and-training-visualizations-with-tensorboard-46368a57fc97 

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import scraper 
from sklearn.preprocessing import LabelEncoder

In [13]:
#getting data ready, selecting classes (most common answers)
df = scraper.hf_dataframe()
qa = df[["question", "answer"]]
qa_clean = qa.applymap(scraper.cleaner) 

top_answers = qa_clean['answer'].value_counts().head(20).index.tolist()

df_filtered = qa_clean[qa_clean['answer'].isin(top_answers)].copy()

le = LabelEncoder()
df_filtered['label'] = le.fit_transform(df_filtered['answer'])

print("Most commonly occuring answers: ", le.classes_)

C:\Users\sjkov\AppData\Local\Temp\ipykernel_10932\870503883.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  qa_clean = qa.applymap(scraper.cleaner)


Most commonly occuring answers:  ['alaska' 'australia' 'california' 'canada' 'chicago' 'china' 'florida'
 'france' 'germany' 'hawaii' 'india' 'ireland' 'italy' 'japan' 'mexico'
 'paris' 'russia' 'south africa' 'spain' 'texas']


In [18]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df_filtered[['question', 'label']])

dataset = hf_dataset.train_test_split(test_size=0.2)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'label', '__index_level_0__'],
        num_rows: 2700
    })
    test: Dataset({
        features: ['question', 'label', '__index_level_0__'],
        num_rows: 675
    })
})


In [19]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

#preprocessing

def preprocess_function(examples):
    return tokenizer(examples["question"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

C:\Users\sjkov\miniforge3\envs\ml\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

Map:   0%|          | 0/675 [00:00<?, ? examples/s]

In [20]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [22]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

to_label = {}
to_id = {}

for index, label_name in enumerate(le.classes_):
    to_label[index] = label_name
    to_id[label_name] = index

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=20,
    id2label=to_label,
    label2id=to_id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#figure out tensorboard setup here

In [ ]:

#bert finetuning here
#experiment with different parameters
#definitely need to mess around with these
args = TrainingArguments(
    output_dir="bert_jeopardy_model",
    eval_steps=50,
    
    logging_strategy="steps",
   
    logging_steps=50,
   
    save_strategy="steps",
    
    save_steps=200,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="tensorboard"
)

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer.train()

In [ ]:
#test

In [ ]:
#compute the metrics (accuracy)